In [2]:
import display.mnistReader as r


In [5]:

reader = r.Reader('mnist/train-labels-idx1-ubyte.gz',
                  'mnist/train-images-idx3-ubyte.gz')


Labels Magic Number:2049
total:60000
Images Magic Number:2051
total:60000
Row/Col: 28/28
